Aluna: Manuela de Lacerda Bezerra Carvalho (mlbc@cesar.school)

In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


## Retrieval Augmented Generation (RAG)

### Carregando Documentos - Loading

In [2]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [6]:
# https://python.langchain.com/v0.2/docs/how_to/#document-loaders
# https://python.langchain.com/v0.2/docs/integrations/document_loaders/

import bs4
from langchain_community.document_loaders import WebBaseLoader

# Filtra o conteúdo da página por uma classe específica
bs4_strainer = bs4.SoupStrainer(class_=("container-wrapper"))

# Carrega o conteúdo da página
loader = WebBaseLoader(
    web_paths=("https://cesar.breezy.hr/p/00f79174d8ad-pesquisador-em-inteligencia-artificial-e-sistemas-distribuidos",),
    bs_kwargs={"parse_only": bs4_strainer},
)

# Carrega o conteúdo da página
docs = loader.load()

len(docs[0].page_content)

4301

### Dividindo Documentos - Splitting/Chunking

In [7]:
# https://python.langchain.com/v0.2/docs/how_to/#text-splitters

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=500, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

6

In [ ]:
print(all_splits[3].page_content)

Requisitos e Qualificações:
Doutorado em áreas correlatas;Compreenda e desenvolva modelos de machine learning e deep learning para resolver desafios complexos de cibersegurança;Conhecimento em frameworks de machine learning como TensorFlow, PyTorch ou scikit-learn para desenvolver modelos preditivos e de detecção de anomalias aplicados à cibersegurança;Entenda a arquitetura distribuída dos sistemas e garanta a integração eficiente de soluções de IA com aplicações em cloud;Habilidade em manipulação e visualização de dados com Pandas, NumPy, Matplotlib e Seaborn para explorar grandes volumes de dados;Experiência com AWS, Google Cloud ou Azure para projetar e implementar infraestruturas escaláveis e resilientes;Familiaridade com Kubernetes e Docker para garantir escalabilidade e resiliência de sistemas distribuídos;Experiência com controle de versão (Git) e repositórios remotos como GitLab;Inglês avançado para leitura, escrita e comunicação, facilitando a colaboração com equipes globais.


### Indexando - Store

In [3]:
!pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 35.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4


In [4]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 42.6 MB/s eta 0:00:00


In [8]:
# https://python.langchain.com/v0.2/docs/how_to/embed_text/

from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

vectorstore = FAISS.from_documents(all_splits, OpenAIEmbeddings())

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("precisa de doutorado para a vaga?")

len(retrieved_docs)

6

In [ ]:
print(retrieved_docs[0].page_content)

Requisitos e Qualificações:
Doutorado em áreas correlatas;Compreenda e desenvolva modelos de machine learning e deep learning para resolver desafios complexos de cibersegurança;Conhecimento em frameworks de machine learning como TensorFlow, PyTorch ou scikit-learn para desenvolver modelos preditivos e de detecção de anomalias aplicados à cibersegurança;Entenda a arquitetura distribuída dos sistemas e garanta a integração eficiente de soluções de IA com aplicações em cloud;Habilidade em manipulação e visualização de dados com Pandas, NumPy, Matplotlib e Seaborn para explorar grandes volumes de dados;Experiência com AWS, Google Cloud ou Azure para projetar e implementar infraestruturas escaláveis e resilientes;Familiaridade com Kubernetes e Docker para garantir escalabilidade e resiliência de sistemas distribuídos;Experiência com controle de versão (Git) e repositórios remotos como GitLab;Inglês avançado para leitura, escrita e comunicação, facilitando a colaboração com equipes globais.


### Buscando e Recuperando Informações - Retrieve

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_template = """Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.

Pergunta: {question}

Contexto: {context}

Resposta:
"""

prompt_template = ChatPromptTemplate.from_template(system_template)

In [ ]:
example_messages = prompt_template.invoke({
    "context": "algum contexto",
    "question": "alguma pergunta"
})

print(example_messages.to_messages())

[HumanMessage(content='Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.\n\nPergunta: alguma pergunta\n\nContexto: algum contexto\n\nResposta:\n', additional_kwargs={}, response_metadata={})]


### Gerando Respostas - Generate

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [ ]:
for chunk in rag_chain.stream("Tem plano de saúde como benefício?"):
    print(chunk, end="", flush=True)

Sim, o CESAR oferece plano de saúde como benefício. Além disso, há também plano odontológico e outros auxílios.

## Exercícios

### Exercício 1
Faça um RAG com um pequeno arquivo de texto, contendo informações que, certamente, a LLM não conheça. Você deverá construir o arquivo e enviar para o ambiente de execução. Escolha a forma de chunking apropriada para o seu documento.

In [ ]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 4.5 MB/s eta 0:00:00


In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_path = (
    "Currículo - Manuela Carvalho.pdf"
)
loader = PyPDFLoader(file_path)
pages = loader.load_and_split()

len(pages)

1

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=500, add_start_index=True
)
all_splits = text_splitter.split_documents(pages)

len(all_splits)

2

In [ ]:
vectorstore = FAISS.from_documents(all_splits, OpenAIEmbeddings())
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("quais são os conhecimentos dessa pessoa?")

len(retrieved_docs)

2

In [ ]:
system_template = """Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.

Pergunta: {question}

Contexto: {context}

Resposta:
"""

prompt_template = ChatPromptTemplate.from_template(system_template)

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [ ]:
for chunk in rag_chain.stream("quais são os conhecimentos dessa pessoa?"):
    print(chunk, end="", flush=True)

Manuela possui conhecimentos em programação, incluindo Java, Spring Boot, JSF, HTML, CSS, Angular, PHP e Laravel, além de inglês intermediário. Ela também tem formação em Análise e Desenvolvimento de Sistemas e especialização em Engenharia e Análise de Dados.